In [1]:
import pandas as pd
import joblib
import numpy as np
from scipy.sparse import hstack, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import os

In [2]:
animes_df = pd.read_parquet("../../data/processed/animes_final.parquet")
animes_df.head(10)

,anime_uid,title,synopsis,aired,episodes,members,popularity,score,genres_list,genre_action,...,genre_emb_40,genre_emb_41,genre_emb_42,genre_emb_43,genre_emb_44,genre_emb_45,genre_emb_46,genre_emb_47,genre_emb_48,genre_emb_49
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"Oct 4, 2015 to Mar 27, 2016",25,489888,141,8.82,"[Comedy, Sports, Drama, School, Shounen]",0,...,-0.325108,0.372583,-0.150133,0.063334,-0.037451,-0.023057,-0.015965,-0.079849,0.069605,0.041172
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Oct 10, 2014 to Mar 20, 2015",22,995473,28,8.83,"[Drama, Music, Romance, School, Shounen]",0,...,0.039412,0.307161,0.010229,0.095036,0.020471,-0.112495,-0.233364,-0.223070,-0.094668,0.010424
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"Jul 7, 2017 to Sep 29, 2017",13,581663,98,8.83,"[Sci-Fi, Adventure, Mystery, Drama, Fantasy]",0,...,0.241236,-0.047506,0.017787,0.183756,0.471296,0.002649,0.130329,-0.239705,-0.322175,-0.608473
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,9.23,"[Action, Military, Adventure, Comedy, Drama, M...",1,...,-0.031031,0.094117,0.084010,-0.000744,0.322104,-0.150647,-0.061140,-0.264919,-0.225668,-0.001621
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"Jan 6, 2017",1,214621,502,8.83,"[Action, Mystery, Supernatural, Vampire]",1,...,0.085453,-0.016847,-0.015368,0.250852,0.117892,-0.276332,0.251657,-0.217527,-0.120232,-0.557234
5,37510,Mob Psycho 100 II,"Shigeo ""Mob"" Kageyama is now maturing and unde...","Jan 7, 2019 to Apr 1, 2019",13,442310,176,8.89,"[Action, Slice of Life, Comedy, Supernatural]",1,...,-0.037127,0.159679,0.233275,-0.244771,0.012024,-0.248264,-0.020235,-0.000815,0.010380,0.002741
6,199,Sen to Chihiro no Kamikakushi,"Stubborn, spoiled, and naïve, 10-year-old Chih...","Jul 20, 2001",1,913212,40,8.90,"[Adventure, Supernatural, Drama]",0,...,0.107310,0.132151,0.278707,0.159719,0.402134,-0.485785,0.263917,-0.151865,-0.420217,0.223778
7,38000,Kimetsu no Yaiba,"Ever since the death of his father, the burden...","Apr 6, 2019 to Sep 28, 2019",26,575037,106,8.92,"[Action, Demons, Historical, Shounen, Supernat...",1,...,0.113221,-0.099797,0.018980,0.003247,0.450094,-0.027240,-0.082962,-0.213881,-0.170457,-0.145498
8,35247,Owarimonogatari 2nd Season,Following an encounter with oddity specialist ...,"Aug 12, 2017 to Aug 13, 2017",7,189944,573,8.93,"[Mystery, Comedy, Supernatural, Vampire]",0,...,-0.078951,0.052401,0.011987,0.156864,-0.289549,-0.147272,0.334582,-0.114548,-0.035374,-0.252579
9,2904,Code Geass: Hangyaku no Lelouch R2,"One year has passed since the Black Rebellion,...","Apr 6, 2008 to Sep 28, 2008",25,992196,27,8.93,"[Action, Military, Sci-Fi, Super Power, Drama,...",1,...,-0.105253,0.141387,0.301835,0.080078,0.368225,-0.211279,0.091863,-0.211304,-0.288166,-0.188470


In [3]:
animes_df = animes_df.sort_values('anime_uid').reset_index(drop=True)
animes_df.head(10)

,anime_uid,title,synopsis,aired,episodes,members,popularity,score,genres_list,genre_action,...,genre_emb_40,genre_emb_41,genre_emb_42,genre_emb_43,genre_emb_44,genre_emb_45,genre_emb_46,genre_emb_47,genre_emb_48,genre_emb_49
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","Apr 3, 1998 to Apr 24, 1999",26,930311,39,8.81,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",1,...,-0.074357,0.365317,0.166373,-0.170352,0.243867,-0.108294,0.012211,-0.281455,-0.240680,-0.315403
1,5,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","Sep 1, 2001",1,223199,475,8.40,"[Action, Drama, Mystery, Sci-Fi, Space]",1,...,-0.059906,0.270419,0.112851,0.045790,0.296700,-0.155649,0.066828,-0.200302,-0.148432,-0.868853
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...","Apr 1, 1998 to Sep 30, 1998",26,460146,158,8.28,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",1,...,-0.077839,0.150744,0.156419,-0.030139,0.318308,-0.054255,-0.021576,-0.317484,-0.462831,-0.193684
3,7,Witch Hunter Robin,Witches are individuals with special powers li...,"Jul 2, 2002 to Dec 24, 2002",26,85182,1278,7.32,"[Action, Magic, Police, Supernatural, Drama, M...",1,...,-0.052032,0.115361,0.145944,0.198596,0.246422,-0.238237,-0.050410,-0.132309,-0.159822,-0.360290
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,"Sep 30, 2004 to Sep 29, 2005",52,12319,3968,7.02,"[Adventure, Fantasy, Shounen, Supernatural]",0,...,0.328435,-0.028888,0.168048,0.166234,0.656518,-0.166169,-0.039358,-0.283561,-0.211242,0.077597
5,15,Eyeshield 21,Sena is like any other shy kid starting high s...,"Apr 6, 2005 to Mar 19, 2008",145,126760,888,8.02,"[Action, Sports, Comedy, Shounen]",1,...,-0.275248,0.315875,0.123684,-0.102866,0.159863,-0.021003,-0.239822,-0.130550,0.223333,-0.068264
6,16,Hachimitsu to Clover,"Yuuta, Takumi, and Shinobu share a six-tatami ...","Apr 15, 2005 to Sep 27, 2005",24,186696,578,8.11,"[Comedy, Drama, Josei, Romance, Slice of Life]",0,...,-0.212132,0.106553,-0.026183,-0.209923,-0.264841,-0.274054,-0.155332,0.027890,-0.100706,0.118697
7,17,Hungry Heart: Wild Striker,Kyosuke Kano has lived under the shadow of his...,"Sep 11, 2002 to Sep 10, 2003",52,18370,3330,7.65,"[Slice of Life, Comedy, Sports, Shounen]",0,...,-0.226213,0.239617,-0.059637,-0.189746,-0.019923,-0.207686,-0.168640,-0.037196,0.462995,0.100305
8,18,Initial D Fourth Stage,Takumi Fujiwara and brothers Keisuke and Ryous...,"Apr 17, 2004 to Feb 18, 2006",24,87746,1228,8.18,"[Action, Cars, Sports, Drama, Seinen]",1,...,-0.210281,0.078994,0.013023,0.133530,0.235673,-0.003192,-0.050306,0.121551,0.059275,-0.484724
9,19,Monster,"Dr. Kenzou Tenma, an elite neurosurgeon recent...","Apr 7, 2004 to Sep 28, 2005",74,459710,159,8.69,"[Drama, Horror, Mystery, Police, Psychological...",0,...,-0.107918,0.085798,0.205727,0.631470,0.044962,0.078458,-0.173369,0.202048,-0.381634,-0.406921


In [4]:
type(animes_df)

pandas.core.frame.DataFrame

In [5]:
try:
    tfidf_vectorizer = joblib.load('../../models/tfidf_vectorizer.joblib')
except FileNotFoundError:
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    tfidf_vectorizer.fit(animes_df['synopsis_clean'].fillna(''))
    joblib.dump(tfidf_vectorizer, '../../models/tfidf_vectorizer.joblib')

In [7]:
tfidf_matrix = tfidf_vectorizer.transform(animes_df['synopsis_clean'])

In [8]:
genre_cols = [f'genre_emb_{i}' for i in range(50)]
genre_vectors = animes_df[genre_cols].values

In [11]:
animes_df.columns[:80]

Index(['anime_uid', 'title', 'synopsis', 'aired', 'episodes', 'members',
       'popularity', 'score', 'genres_list', 'genre_action', 'genre_adventure',
       'genre_cars', 'genre_comedy', 'genre_dementia', 'genre_demons',
       'genre_drama', 'genre_ecchi', 'genre_fantasy', 'genre_game',
       'genre_harem', 'genre_hentai', 'genre_historical', 'genre_horror',
       'genre_josei', 'genre_kids', 'genre_magic', 'genre_martial_arts',
       'genre_mecha', 'genre_military', 'genre_music', 'genre_mystery',
       'genre_parody', 'genre_police', 'genre_psychological', 'genre_romance',
       'genre_samurai', 'genre_school', 'genre_sci_fi', 'genre_seinen',
       'genre_shoujo', 'genre_shoujo_ai', 'genre_shounen', 'genre_shounen_ai',
       'genre_slice_of_life', 'genre_space', 'genre_sports',
       'genre_super_power', 'genre_supernatural', 'genre_thriller',
       'genre_vampire', 'genre_yaoi', 'genre_yuri', 'source', 'n_genres',
       'rare_genre_flag', 'genre_entropy', 'synopsis_cle

In [10]:
numeric_cols = [
    'score_scaled', 'episodes_scaled', 'members_log_scaled',
    'popularity_log_scaled', 'anime_age', 'genre_action', 'genre_adventure',
    'genre_cars', 'genre_comedy', 'genre_dementia', 'genre_demons',
    'genre_drama', 'genre_ecchi', 'genre_fantasy', 'genre_game',
    'genre_harem', 'genre_hentai', 'genre_historical', 'genre_horror',
    'genre_josei', 'genre_kids', 'genre_magic', 'genre_martial_arts',
    'genre_mecha', 'genre_military', 'genre_music', 'genre_mystery',
    'genre_parody', 'genre_police', 'genre_psychological', 'genre_romance',
    'genre_samurai', 'genre_school', 'genre_sci_fi', 'genre_seinen',
    'genre_shoujo', 'genre_shoujo_ai', 'genre_shounen', 'genre_shounen_ai',
    'genre_slice_of_life', 'genre_space', 'genre_sports',
    'genre_super_power', 'genre_supernatural', 'genre_thriller',
    'genre_vampire', 'genre_yaoi', 'genre_yuri'
]

In [11]:
numeric_features = animes_df[numeric_cols].fillna(0).values

In [14]:
combined_features = hstack([
    csr_matrix(tfidf_matrix),
    csr_matrix(genre_vectors),
    csr_matrix(numeric_features)
]).tocsr()

In [15]:
similarity_matrix = cosine_similarity(combined_features)

In [16]:
np.savez_compressed('anime_similarity_matrix.npz', matrix=similarity_matrix.astype(np.float16))

In [18]:
uid_to_idx_mapper = pd.Series(animes_df.index, index=animes_df.anime_uid)
uid_to_idx_mapper.to_pickle('cb_uid_to_idx_mapper.pkl')

In [19]:
similarity_matrix = np.load('anime_similarity_matrix.npz')['matrix'].astype(np.float32)

In [20]:
uid_to_idx = pd.read_pickle('cb_uid_to_idx_mapper.pkl')

In [21]:
idx_to_uid = pd.Series(uid_to_idx.index, index=uid_to_idx.values)

In [22]:
animes = pd.read_parquet('../../data/processed/animes_final.parquet')[['anime_uid', 'title']]
animes.set_index('anime_uid', inplace=True)

In [257]:
def get_recommendations(anime_uid: int, n: int = 10):
    if anime_uid not in uid_to_idx:
        return {"error": f"Anime with ID {anime_uid} not found."}

    anime_idx = uid_to_idx[anime_uid]

    similarity_scores = list(enumerate(similarity_matrix[anime_idx]))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    top_scores = similarity_scores[1:n + 1]

    top_anime_indices = [i[0] for i in top_scores]
    top_anime_scores = [i[1] for i in top_scores]

    recommended_uids = idx_to_uid.loc[top_anime_indices].tolist()

    results = []
    for uid in recommended_uids:
        results.append(uid)

    return results

In [258]:
test_anime_id = 20

In [260]:
recommendations = get_recommendations(anime_uid=test_anime_id, n=10)

recommendations

[1604, 1482, 6702, 918, 269, 813, 6033, 249, 121, 527]

In [261]:
profiles_df = pd.read_parquet('../../data/processed/profiles_cleaned_final.parquet')

In [195]:
profiles_df["favorites_list"] = profiles_df["favorites_list"].apply(lambda lst: [int(x) for x in lst])
type(profiles_df.iloc[0, 9][0])

int

In [262]:
profiles_df.head(10)

,gender,birthday,favorites_anime,gender_male,gender_female,gender_unknown,birth_year,age,age_missing,favorites_list,num_favorites,has_favorites,uid
0,male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",1,0,0,1994.0,31.0,0,"[33352, 25013, 5530, 33674, 1482, 269, 18245, ...",20,1,255938
1,female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",0,1,0,2000.0,25.0,0,"[11061, 31964, 853, 20583, 918, 9253, 34599, 3...",10,1,259117
2,unknown,None,"['918', '2904', '11741', '17074', '23273', '32...",0,0,1,NaN,NaN,1,"[918, 2904, 11741, 17074, 23273, 32281, 9989, ...",9,1,253664
3,male,Sep 5,"['5680', '849', '2904', '3588', '37349']",1,0,0,NaN,NaN,1,"[5680, 849, 2904, 3588, 37349]",5,1,8254
4,male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",1,0,0,1999.0,26.0,0,"[4181, 7791, 9617, 5680, 2167, 4382, 849, 235,...",10,1,291149
5,unknown,None,"['5114', '4898', '2904', '1575', '1482']",0,0,1,NaN,NaN,1,"[5114, 4898, 2904, 1575, 1482]",5,1,10046
6,unknown,"May 29, 1996","['338', '322', '440', '199', '28223', '12815',...",0,0,1,1996.0,29.0,0,"[338, 322, 440, 199, 28223, 12815, 2800, 18679...",10,1,247454
7,male,"Jan 1, 1985","['11061', '30', '6594', '28701', '10087', '674...",1,0,0,1985.0,40.0,0,"[11061, 30, 6594, 28701, 10087, 6746, 918, 153...",10,1,140903
8,male,"Jul 29, 1992","['846', '2904', '5114', '2924', '72']",1,0,0,1992.0,33.0,0,"[846, 2904, 5114, 2924, 72]",5,1,23791
9,unknown,None,[],0,0,1,NaN,NaN,1,[],0,0,25115


In [265]:
profiles_df.columns

Index(['gender', 'birthday', 'favorites_anime', 'gender_male', 'gender_female',
       'gender_unknown', 'birth_year', 'age', 'age_missing', 'favorites_list',
       'num_favorites', 'has_favorites', 'uid'],
      dtype='object')

In [7]:
animes_df['genres_str'] = animes_df['genres_list'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)


def create_soup(row):
    # Combine the lower-cased Synopsis and Genre strings
    return (row['synopsis'] + ' ' + row['genres_str']).lower()


# 3. Apply the function to create the new feature column
animes_df['soup'] = animes_df.apply(create_soup, axis=1)

animes_df.head()

,anime_uid,title,synopsis,aired,episodes,members,popularity,score,genres_list,genre_action,...,genre_emb_42,genre_emb_43,genre_emb_44,genre_emb_45,genre_emb_46,genre_emb_47,genre_emb_48,genre_emb_49,genres_str,soup
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever...","Apr 3, 1998 to Apr 24, 1999",26,930311,39,8.81,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",1,...,0.166373,-0.170352,0.243867,-0.108294,0.012211,-0.281455,-0.240680,-0.315403,Action Adventure Comedy Drama Sci-Fi Space,"in the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Another day, another bounty—such is the life o...","Sep 1, 2001",1,223199,475,8.40,"[Action, Drama, Mystery, Sci-Fi, Space]",1,...,0.112851,0.045790,0.296700,-0.155649,0.066828,-0.200302,-0.148432,-0.868853,Action Drama Mystery Sci-Fi Space,"another day, another bounty—such is the life o..."
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0...","Apr 1, 1998 to Sep 30, 1998",26,460146,158,8.28,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",1,...,0.156419,-0.030139,0.318308,-0.054255,-0.021576,-0.317484,-0.462831,-0.193684,Action Sci-Fi Adventure Comedy Drama Shounen,"vash the stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,Witches are individuals with special powers li...,"Jul 2, 2002 to Dec 24, 2002",26,85182,1278,7.32,"[Action, Magic, Police, Supernatural, Drama, M...",1,...,0.145944,0.198596,0.246422,-0.238237,-0.050410,-0.132309,-0.159822,-0.360290,Action Magic Police Supernatural Drama Mystery,witches are individuals with special powers li...
4,8,Bouken Ou Beet,It is the dark century and the people are suff...,"Sep 30, 2004 to Sep 29, 2005",52,12319,3968,7.02,"[Adventure, Fantasy, Shounen, Supernatural]",0,...,0.168048,0.166234,0.656518,-0.166169,-0.039358,-0.283561,-0.211242,0.077597,Adventure Fantasy Shounen Supernatural,it is the dark century and the people are suff...


In [8]:
tf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=5,
    stop_words='english'
)

tfidf_matrix = tf.fit_transform(animes_df['soup'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(tfidf_matrix.shape)
print(cosine_sim.shape)

(16368, 18326)
(16368, 16368)


In [12]:
combined_features = hstack([
    csr_matrix(tfidf_matrix),
    csr_matrix(numeric_features)
]).tocsr()

In [13]:
similarity_matrix = cosine_similarity(combined_features)

In [14]:
uid_to_idx_mapper = pd.Series(animes_df.index, index=animes_df.anime_uid)

In [17]:
def get_recommendations(title, cosine_sim=similarity_matrix, df=animes_df, mapper=None):
    try:
        # Get the index of the anime that matches the title
        idx = df[df['title'] == title].index[0]
    except IndexError:
        return "Anime not found in the dataset."

    # Get the raw similarity scores for that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Prepare data for hybrid scoring
    anime_ratings = df['score'].values

    # Define weights for the hybrid model
    W_SIMILARITY = 0.7
    W_RATING = 0.3

    hybrid_scores = []
    # and removed the invalid [1] index.
    for i, sim_score_value in sim_scores:
        # Normalize the 0-10 score to a 0-1 range
        normalized_rating = anime_ratings[i] / 10

        # Calculate weighted score
        # The scalar similarity value is used directly here
        weighted_score = (W_SIMILARITY * sim_score_value) + (W_RATING * normalized_rating)

        # Store the index and the new score
        hybrid_scores.append((i, weighted_score))

    # Sort the animes based on the new Hybrid Score
    sorted_scores = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the top 10 most similar animes (excluding itself)
    anime_indices = [i[0] for i in sorted_scores[1:11]]

    # Return the top recommended anime titles
    return df[['title', 'score']].iloc[anime_indices]

In [275]:
joblib.dump(similarity_matrix, 'cb_similarity_matrix.pkl')

# Save the UID-to-Index mapper (you already had this line, ensure it runs)
uid_to_idx_mapper = pd.Series(animes_df.index, index=animes_df.anime_uid)
uid_to_idx_mapper.to_pickle('cb_uid_to_idx_mapper_2.pkl')

In [32]:
get_recommendations('One Piece')

,title,score
0,Cowboy Bebop,8.81
149,Mononoke Hime,8.77
749,Ginga Eiyuu Densetsu,9.03
3543,Clannad: After Story,8.97
412,Howl no Ugoku Shiro,8.71
246,Hajime no Ippo,8.77
227,Great Teacher Onizuka,8.71
11,One Piece,8.53
2660,Code Geass: Hangyaku no Lelouch R2,8.93
6009,Gintama',9.04


In [19]:
def jaccard_similarity(list1, list2):
    """Calculates Jaccard Similarity between two lists."""
    s1 = set(list1)
    s2 = set(list2)
    # Avoid division by zero if both lists are empty, though unlikely in this context
    union_size = len(s1.union(s2))
    return len(s1.intersection(s2)) / union_size if union_size > 0 else 0.0

In [21]:
def evaluate_top_n_diversity(model_function, test_animes, n=10):
    all_recommendations = []
    for anime in test_animes:
        recs_df = model_function(anime)
        if isinstance(recs_df, pd.DataFrame):
            # Take only the top N titles
            all_recommendations.append(recs_df['title'].head(n).tolist())

    # Calculate pairwise Jaccard similarity for all recommendation lists
    jaccard_scores = []
    for i in range(len(all_recommendations)):
        for j in range(i + 1, len(all_recommendations)):
            # Only compare lists that aren't empty
            if all_recommendations[i] and all_recommendations[j]:
                score = jaccard_similarity(all_recommendations[i], all_recommendations[j])
                jaccard_scores.append(score)

    if not jaccard_scores:
        print("Could not generate enough unique recommendation lists to calculate diversity.")
        return 0.0

    # The diversity score is 1 - average similarity
    avg_jaccard = np.mean(jaccard_scores)
    diversity_score = 1 - avg_jaccard

    print(f"Average Jaccard Similarity between recommendation lists: {avg_jaccard:.2f}")
    print(f"Model Diversity Score (1 - Jaccard): {diversity_score:.2f}")

    return diversity_score

In [22]:
def evaluate_average_similarity_score(cosine_sim_matrix, test_animes, df):
    all_similarity_scores = []

    for anime_title in test_animes:
        try:
            # Get the index of the anime
            idx = df[df['title'] == anime_title].index[0]

            # Get the top 10 similar animes based on the raw cosine similarity
            sim_scores = list(enumerate(cosine_sim_matrix[idx]))
            sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Get the top 10 scores (excluding self-similarity at index 0)
            top_10_scores = [score[1] for score in sorted_scores[1:11]]
            all_similarity_scores.extend(top_10_scores)

        except IndexError:
            # Skip animes not found in the DataFrame
            continue

    if not all_similarity_scores:
        print("No similarity scores found for the test set.")
        return 0.0

    avg_score = np.mean(all_similarity_scores)
    print(f"Average Similarity Score of all recommended items: {avg_score:.2f}")
    return avg_score

In [24]:
import random

all_titles = animes_df['title'].tolist()
# Check to ensure we have enough titles
sample_size = min(len(all_titles), 100)
test_animes = random.sample(all_titles, sample_size)

print("--- Running Evaluation ---")
print(f"Evaluating the model on {len(test_animes)} randomly selected animes.")

--- Running Evaluation ---
Evaluating the model on 100 randomly selected animes.


In [25]:
print("\n[1] Evaluating Recommendation Diversity...")
diversity_score = evaluate_top_n_diversity(get_recommendations, test_animes)


[1] Evaluating Recommendation Diversity...
Average Jaccard Similarity between recommendation lists: 0.27
Model Diversity Score (1 - Jaccard): 0.73


In [26]:
print("\n[2] Evaluating Average Similarity Score...")
avg_sim_score = evaluate_average_similarity_score(similarity_matrix, test_animes, animes_df)


[2] Evaluating Average Similarity Score...
Average Similarity Score of all recommended items: 0.96


In [27]:
joblib.dump(similarity_matrix, 'cb_similarity_matrix.pkl')

# Save the UID-to-Index mapper (you already had this line, ensure it runs)
uid_to_idx_mapper = pd.Series(animes_df.index, index=animes_df.anime_uid)
uid_to_idx_mapper.to_pickle('cb_uid_to_idx_mapper_2.pkl')

In [ ]:
np.savez_compressed('anime_similarity_matrix.npz', matrix=similarity_matrix.astype(np.float16))

In [44]:
animes_df[animes_df['score'] > 5.0][['title', 'score']]

,title,score
0,Cowboy Bebop,8.81
1,Cowboy Bebop: Tengoku no Tobira,8.40
2,Trigun,8.28
3,Witch Hunter Robin,7.32
4,Bouken Ou Beet,7.02
...,...,...
16363,Hige wo Soru. Soshite Joshikousei wo Hirou.,6.48
16364,Enen no Shouboutai: Ni no Shou,6.37
16365,Shin Chuuka Ichiban! 2,6.35
16366,Shin Chuuka Ichiban! 2,6.35


In [48]:
get_recommendations('Hajime no Ippo')

,title,score
246,Hajime no Ippo,8.77
183,Sen to Chihiro no Kamikakushi,8.90
0,Cowboy Bebop,8.81
227,Great Teacher Onizuka,8.71
4059,Hajime no Ippo: New Challenger,8.69
149,Mononoke Hime,8.77
26,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,8.75
6009,Gintama',9.04
3543,Clannad: After Story,8.97
155,Slam Dunk,8.53


Overall Summary & Conclusion

**Objective:** Build and evaluate a content-based (hybridized) anime recommendation pipeline leveraging textual synopsis + genres, engineered numeric/genre features, and a simple rating-aware re-ranking.

 **Pipeline Recap:**
 1. Loaded processed anime dataset (`animes_final.parquet`) and ensured stable UID→index mapping.
 2. Engineered feature blocks:
    - TF-IDF on synopsis (clean) and later a combined "soup" (synopsis + genres) with unigrams & bigrams.
    - Learned / precomputed genre embedding vectors (50 dims) + one‑hot genre indicators.
    - Scaled numeric meta features (score, episodes, members, popularity, age, etc.).
 3. Constructed sparse composite feature matrix via horizontal stacking (text + genre embedding + numeric).
 4. Computed cosine similarity matrix (persisted both compressed `.npz` and a Joblib pickle variant).
 5. Implemented two recommendation variants:
    - Pure similarity on combined features by UID.
    - Hybrid title-based function weighting similarity (0.7) + normalized rating (0.3).
 6. Added evaluation utilities:
    - Diversity: 1 - mean pairwise Jaccard between top-N lists across a random sample.
    - Average similarity score of recommended items (proxy for how "tight" / redundant lists are).

 **Interpreting Evaluation Metrics:**
 - `diversity_score` (higher is better): Indicates how distinct recommendation lists are across different seed titles. Extremely low values suggest homogenization / popularity bias.
 - `avg_sim_score` (moderate is ideal): Very high may imply overfitting to near-duplicates; very low may indicate noisy / unfocused neighborhoods.

 **Strengths Observed:**
 - Flexible fusion of heterogeneous signals (textual semantics, genres, numeric metadata).
 - Lightweight, fully offline computation (cosine on sparse matrix) suitable for fast lookup.
 - Rating-aware hybridization improves practical relevance vs. raw similarity alone.

 **Limitations / Risks:**
 - No personalization yet (user history / favorites only implicitly inspected, not modeled).
 - Rating weighting is static; could be tuned or learned (e.g., via validation objective).
 - Genre embeddings origin/quality not validated here (could analyze variance / clustering quality).
 - Cold start for brand-new anime still depends entirely on provided synopsis & genres.
 - Diversity metric is simplistic (Jaccard on titles); could expand to intra-list diversity (e.g., topic dispersion) and coverage.

 **Potential Improvements / Next Steps:**
 1. Introduce user-personalized re-ranking (e.g., aggregate favorite feature centroid → similarity, or a hybrid with collaborative filtering / implicit feedback).
 2. Calibrate weights (similarity vs. rating) via grid search optimizing a composite objective (diversity + average relevance proxy).
 3. Add popularity dampening (e.g., log penalty) to reduce mainstream dominance.
 4. Persist a top-k sparse similarity index (ANN via Faiss / ScaNN) for scalability if dataset grows.
 5. Evaluate with additional metrics: coverage, novelty, intra-list diversity, serendipity proxies.
 6. Consider semantic embeddings (e.g., Sentence-BERT on synopsis) to complement TF-IDF n-grams.
 7. Add guard-rails: filter near-identical sequels / alternate cuts when user wants variety.

**How to Use (Current Artifacts):**
- Load `cb_similarity_matrix.pkl` (or compressed `.npz`) and `cb_uid_to_idx_mapper_2.pkl` for UID-based lookups.
- Call `get_recommendations(title)` for hybrid scoring (ensure title exists) or adapt to accept UID.

test new git_comm
